# Experiment Tracking

## Logging Things Manually

In [ ]:
import mlflow
# parameter

mlflow.log_param('name', 'value')
mlflow.log_params({'name1': 'value1'})
mlflow.log_metric('metric1', 1)
mlflow.log_metrics('Dictionary of metrics')

In [ ]:
# logging a model manually
# from https://mlflow.org/docs/latest/getting-started/intro-quickstart/index.html
# Set our tracking server uri for logging
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

# Create a new MLflow Experiment
mlflow.set_experiment("MLflow Quickstart")

# Start an MLflow run
with mlflow.start_run() as run:
    run_id = run.run_info.run_id
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
    )


## Metadata
All MLflow runs are logged to the active experiment, which can be set using any of the following ways:

Use the mlflow.set_experiment() command.

Use the experiment_id parameter in the mlflow.start_run() command.

Set one of the MLflow environment variables MLFLOW_EXPERIMENT_NAME or MLFLOW_EXPERIMENT_ID.

In [ ]:
# searching runs
mlflow.search_runs(experiment_ids: Optional[List[str]] = None, filter_string: str = '', run_view_type: int = 1, max_results: int = 100000, order_by: Optional[List[str]] = None, output_format: str = 'pandas', search_all_experiments: bool = False, experiment_names: Optional[List[str]] = None) → Union[List[Run], pandas.DataFrame]

### Model Signatures

In [ ]:
# updating existing model signature
import pandas as pd
from sklearn import datasets
import mlflow
from mlflow.models.model import get_model_info
from mlflow.models import infer_signature, set_signature

# load the logged model
model_uri = f"runs:/{run.info.run_id}/iris_rf"
model = mlflow.pyfunc.load_model(model_uri)

# construct the model signature from test dataset
X_test, _ = datasets.load_iris(return_X_y=True, as_frame=True)
signature = infer_signature(X_test, model.predict(X_test))

# set the signature for the logged model
set_signature(model_uri, signature)

# now when you load the model again, it will have the desired signature
assert get_model_info(model_uri).signature == signature

In [10]:
# using predict parameters as part of the model signature
import mlflow
from mlflow.models import infer_signature
from xgboost import XGBClassifier 
from sklearn.datasets import make_classification

X, y = make_classification(random_state=42)

xgb = XGBClassifier()
xgb.fit(X, y)

xgb.predict(X, iteration_range=(1, 3)) # iteration range

signature = infer_signature(X, params={'iteration_range': [2, 5]})

with mlflow.start_run() as run:
  run_id = run.info.run_id
  model_info = mlflow.sklearn.log_model(
    sk_model=xgb, 
    artifact_path='xgb_model',
    signature=signature
  )

In [9]:
loaded_model = mlflow.sklearn.load_model(model_info.model_uri)
loaded_model.predict(X, iteration_range=(10, 12))

array([0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0])

### Model Input Example

Including an input example while logging a model offers dual benefits. Firstly, it aids in inferring the model’s signature. Secondly, and just as importantly, it validates the model’s requirements. This input example is utilized to execute a prediction using the model that is about to be logged, thereby enhancing the accuracy in identifying model requirement dependencies. It is highly recommended to always include an input example along with your models when you log th

In [ ]:
# example code, called with logging the model
input_example = {
    "sepal length (cm)": 5.1,
    "sepal width (cm)": 3.5,
    "petal length (cm)": 1.4,
    "petal width (cm)": 0.2,
}
mlflow.sklearn.log_model(..., input_example=input_example)

# TODO:

disect https://mlflow.org/docs/latest/python_api/mlflow.shap.html 